In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from copy import deepcopy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
#from orion.contrib.envs import load_env
#load_env()
import boto3
#from orion.sources import S3Source
aws_bucket = 'kilimanjaro-prod-datalake'
s3 = boto3.client('s3')

In [2]:
test = pd.read_csv('transaction_tidy.csv', index_col=0)
test.head(2)

/home/emmaranderson7/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,year,month,dayofweek,sales_type,gsales_perunit
0,2139607,S0016-8920840,2015-08-30,13:00:00,2015-08-30,220c0261086d7d2c254a9ba8d9339e6c15cb2792,United Kingdom,S61,90956,Clearance,0.0,1,58.16,15.9233,2015,8,Sunday,S0,58.16
1,2842275,S0016-9272310,2016-03-10,15:19:00,2016-03-10,375083e129410daa79593eb12317e1a3ead57f59,United Kingdom,ML2,87518,Clearance,0.0,1,63.94,10.9850,2016,3,Thursday,S0,63.94


In [3]:
test = test[['customerkey','orderdate','itemnumber','unitssold','grosssales','gsales_perunit']]

In [4]:
orders = test.groupby(
   ['customerkey','orderdate']
).agg(
    { 
         'grosssales': "sum",
    }
).round(decimals=2)
orders= orders.reset_index()

In [5]:
freq = orders.groupby(
   ['customerkey']
).agg(
    {
         'orderdate':lambda x: x.nunique(),  
    }
).round(decimals=2)
freq= freq.reset_index()
freq

,customerkey,orderdate
0,0000010177de5b4048370276b036ff3ba06b7ae6,1
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,1
2,0000089aee56e015b7de440137bbd90baf2b6039,2
3,000009008567d4740aec3d882e67fe4d2d74eb39,1
4,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2
...,...,...
2280360,ffffdf2d6e78c31b7669022a96d6e8ab0f076404,1
2280361,ffffe0a8c1a504232cebca86d2be5e3f59cb9f92,4
2280362,ffffe0b8b360367113c4f1e3d88a9ceb14c031aa,2
2280363,fffff36d72733a953ace2a324b8b231502186204,1


In [10]:
orders = test.groupby(
   ['customerkey','orderdate']
).agg(
    {
         'itemnumber':lambda x: x.nunique(),  
         'unitssold': "sum",
         'grosssales': "sum",
         'gsales_perunit':"mean"
    }
).round(decimals=2)
orders= orders.reset_index()

KeyboardInterrupt: 

In [ ]:
orders = orders.sort_values("grosssales", ascending=False)
orders.head()

In [ ]:
freq = orders.groupby(
   ['customerkey']
).agg(
    {
         'orderdate':lambda x: x.nunique(),  
         'unitssold': "sum",
         'grosssales': "sum",
         'gsales_perunit':"mean"
    }
).round(decimals=2)
freq= freq.reset_index()
freq

In [ ]:
freq = freq.sort_values("orderdate", ascending=False)
freq = freq.rename(columns={"orderdate": "orders"})
freq.head()

In [ ]:
freq.to_csv('customer_orderfreq.csv')

In [ ]:
items = pd.read_csv('units_sorted.csv', index_col=0)
items.head()